In [1]:
%autosave 0
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler

# Téléchargement de la liste de stop words en français.
nltk.download('stopwords')

# Importations de la liste de stop words en français.
from nltk.corpus import stopwords
stop_words_french = set(stopwords.words('french'))

Autosave disabled


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\micka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Lecture du fichier.
df_test = pd.read_excel("df_clean.xlsx")
df_test.head(10)

,Client,Commentaire,Note,Sentiment
0,marc lesterpt,Je commande mes pneus de motos et de voitures ...,5,Positif
1,Séverine M,"BonjourLe site est génial, très bien expliqué ...",5,Positif
2,PINTO NUNO,"Tout était parfait, timing parfait dans les da...",5,Positif
3,client,Pneus d'un tres bon rapport qualité prix. Les ...,5,Positif
4,Vignaux,"Bon suivi de la commande,les questions sont im...",4,Positif
5,cliente juliema,Ce service sur place est d un grand secours. R...,5,Positif
6,cliente,La commande sur internet c'est très bien passé...,5,Positif
7,Manu,"RAS, le rapport qualité prix est très correct....",5,Positif
8,Xavier,Fidèle depuis de nombreuses années j aï toujou...,5,Positif
9,Sonia Nutrition,M'a été recommandé. Meilleur rapport qualité p...,5,Positif


In [3]:
# Suppression des caractères spéciaux, chiffres, valeurs manquantes et texte en minuscules.
df_test['Sentiment'] = df_test['Sentiment'].replace('Neutre', 'Positif')
df_test['Commentaire'] = df_test['Commentaire'].str.replace('[^a-zA-Z\s]', '').str.lower()
df_test['Commentaire'] = df_test['Commentaire'].fillna('') 
df_test.head(10)

C:\Users\micka\AppData\Local\Temp\ipykernel_5684\2210573010.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['Commentaire'] = df_test['Commentaire'].str.replace('[^a-zA-Z\s]', '').str.lower()


,Client,Commentaire,Note,Sentiment
0,marc lesterpt,je commande mes pneus de motos et de voitures ...,5,Positif
1,Séverine M,bonjourle site est gnial trs bien expliqu pour...,5,Positif
2,PINTO NUNO,tout tait parfait timing parfait dans les date...,5,Positif
3,client,pneus dun tres bon rapport qualit prix les mei...,5,Positif
4,Vignaux,bon suivi de la commandeles questions sont imm...,4,Positif
5,cliente juliema,ce service sur place est d un grand secours ri...,5,Positif
6,cliente,la commande sur internet cest trs bien passe r...,5,Positif
7,Manu,ras le rapport qualit prix est trs correct tou...,5,Positif
8,Xavier,fidle depuis de nombreuses annes j a toujours ...,5,Positif
9,Sonia Nutrition,ma t recommand meilleur rapport qualit prix et...,5,Positif


In [4]:
# Division du DataFrame en ensembles d'entraînement et de test.
X_train, X_test, y_train, y_test = train_test_split(df_test['Commentaire'], df_test['Sentiment'], test_size=0.2, random_state=42)

In [5]:
# Appliquer une vectorisation de texte via CountVectorizer avec stop words en français.
vectorizer = CountVectorizer(stop_words=list(stop_words_french))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [6]:
# Appliquer l'oversampling sur l'ensemble d'entraînement seulement.
oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_vec, y_train)

In [7]:
# Créer un classificateur clf en utilisant la méthode GradientBoostingClassifier.
clf = GradientBoostingClassifier()

# Entraîner le classificateur sur l'ensemble d'entraînement resamplé.
clf.fit(X_train_resampled, y_train_resampled)

GradientBoostingClassifier()

In [8]:
# (g) Faire des prédictions sur l'ensemble de test original (non vectorisé)
X_test_resampled, y_test_resampled = oversampler.fit_resample(X_test_vec, y_test)
y_pred = clf.predict(X_test_resampled)

In [9]:
# Affichage du rapport de classification du modèle Gradient Boosting.
print("Rapport de classification :\n", classification_report(y_test_resampled, y_pred))

Rapport de classification :
               precision    recall  f1-score   support

     Négatif       0.87      0.88      0.88     39181
     Positif       0.88      0.87      0.87     39181

    accuracy                           0.87     78362
   macro avg       0.87      0.87      0.87     78362
weighted avg       0.87      0.87      0.87     78362



In [10]:
# Calcul et affichage de la matrice de confusion.
confusion_matrix = pd.crosstab(y_test_resampled, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

Classe prédite,Négatif,Positif
Classe réelle,,
Négatif,34503,4678
Positif,5130,34051
